# Part 0 - Data Preprocessing

In [ ]:
#Importing the Keras Libraries and Packages
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
tf.__version__

'2.2.0'

Preprocessing the Training Set

In [ ]:
#we got the following code from keras api https://keras.io/api/preprocessing/image/
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#we're just performing augmentations to the image such as flipping and shearing so that we can train better on our training set

training_set = train_datagen.flow_from_directory(
        'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
#target size is just the final image size, so we picked 64x64 pixels cuz it makes the training not that long
#classmode is binary cuz its either dog or cat, but categorical is the other option if we have more


Found 8000 images belonging to 2 classes.


Preprocessing the Test Set

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) #so we don't perform the shearing and flipping and zooming on the test set, cuz we're not training with it. We just feed it into our model as is, but we are rescaling as we must do with a normal ANN, cuz we feature scaled it

test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')


Found 2000 images belonging to 2 classes.


# Part 1 - Building the CNN

In [ ]:
#Initializing the CNN
classifier = tf.keras.models.Sequential()

### 1.1 Convolution

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (64,64,3), activation = 'relu')) #we chose 32 feature detectors (filters), meaning we'll create 32 layers of feature maps. 32 is the standard default value to go with, so we just doubled it. Our feature detector matrix will have a size (kernel) size of 3x3
#the input_shape is saying we have 3 channels because we have a coloured image (it'd be 1 for a bw image), and 64,64 are the dimensions of the 2D array

###1.2 Max Pooling

In [ ]:
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides = 2))
#we picked 2x2 as our max pooling feature map, and it will pick the maximum value for each 2x2 pool every 2 strides. This ultimately reduced the size, making our model easier to process

###1.2.1 Adding a Second Convolutional Layer to Improve Accuracy

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')) #we chose 64 feature detectors, meaning we'll create 64 layers of feature maps. We doubled it from previous because it'll help get an even more accurate result. Our feature detector matrix will have a size (kernel) size of 3x3
#Didn't near the input shape above cuz we already did it the first time
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides = 2))


###1.3 Flattening

In [ ]:
classifier.add(tf.keras.layers.Flatten())
#Now we vectorize the Max Pooling Feature Map, to get a spacial properties/structure of the pixels and then we can add that into our ANN as numbers

###1.4 Full Connection

In [ ]:
classifier.add(tf.keras.layers.Dense(units = 128, activation = 'relu')) #we have 128 determined by experiments and to select a power of 2
classifier.add(tf.keras.layers.Dropout(0.4))

classifier.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) #we have 1 cuz we only have 1 overall output prediction, and sigmoid is for binary outputs

###1.5 Compiling the CNN

In [ ]:
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy']) #so here we're not using mse as the loss cuz its a binary classification
#use loss = categorical_crossentropy if we have more than 2 outcomes

# Part 2 - Fitting the CNN

Adding a Checkpoint

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("classifier_CatOrDog_1CP.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto', period=1)

'\nclassifier.fit(\n        training_set,\n        steps_per_epoch=8000,\n        epochs=25,\n        validation_data=test_set,\n        validation_steps=2000,\n        verbose=1)\n'

Adding Early Stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 25) #here our mode is min, because we're trying to minimize the loss. if our metric was accuracy, we'd use max instead as the mode. patience = 5 means we wait 5 epochs until it stops 


In [ ]:
classifier.fit(
        x = training_set,
        steps_per_epoch=8000/32,
        epochs=100,
        validation_data=test_set,
        validation_steps = 2000/32,
        callbacks=[checkpoint, es])
#32 is the batch size

classifier.save_weights('CatorDog_CNN.h5')
classifier.save('CatorDog_CNN.h5')

# Part 3 - Making New Predictions

In [ ]:
import numpy as np
from keras.preprocessing import image 

test_image = image.load_img('/d1.jpg', target_size=(64, 64)) #making our image 64,64 to fit into our model

test_image = image.img_to_array(test_image) #adding the 3 dimension later
test_image = np.expand_dims(test_image, axis = 0) #adding the 4th dimension for what the predict method expects
result = classifier.predict(test_image)

if result[0][0]==1:
  prediction = 'dog'
else:
  prediction = 'cat'
  

print(training_set.class_indices)
print(prediction, result)

{'cats': 0, 'dogs': 1}
dog
